In [3]:
%matplotlib inline

# ALE meta-analysis

In [4]:
import os
from os import listdir
from os.path import isfile, join
import nibabel as nib
from pathlib import Path

import matplotlib.pyplot as plt
from nilearn.plotting import plot_stat_map
from nimare.io import convert_sleuth_to_dataset
from nimare.utils import get_resource_path

from nimare.correct import FWECorrector
from nimare.meta.cbma import ALE
import pickle
import csv

In [14]:
_dset_folder = r'../data_db/dataset_ale/'
_nii_folder = r'../data_db/nifti-ale/'
_res_folder = r'../data_db/res/'
_mask_img_folder = r'../data_db/nifti-ale/templates'
_base_img_filename = r'MNI152_2x2x2_brainmask.nii.gz'

## ALE Working Memory: visual task stimulus and task modality n-task

In [15]:
dsetfiles = {Path(f).stem: f for f in listdir(_dset_folder) if isfile(join(_dset_folder, f))}

In [16]:
files = {k: os.path.join(os.path.abspath(_dset_folder), file_name) for k, file_name in dsetfiles.items()}
dsets = {k: convert_sleuth_to_dataset(f) for k, f in files.items()}

## Apply PFC mask

In [19]:
base_mg = nib.load(os.path.join(os.path.abspath(_mask_img_folder), _base_img_filename))

### Obtain non null coordinates

In [22]:
def get_non_null_nifti_val(nifti):
    pts_lst = []
    for i in range(nifti.get_fdata().shape[0]):
        for j in range(nifti.get_fdata().shape[1]):
            for k in range(nifti.get_fdata().shape[2]):
                if nifti.get_fdata()[i, j, k] != 0: pts_lst.append([i, j, k])
    return pts_lst

In [23]:
pts_list_conj = get_non_null_nifti_val(base_mg)

In [47]:
from scipy.interpolate import interp1d
import util.function_conv_mni as util_conv
from IPython.display import display
import importlib
importlib.reload(util_conv)

<module 'util.function_conv_mni' from 'D:\\scul\\uaic-ub\\licenta\\code\\util\\function_conv_mni.py'>

In [66]:
#Interpolation
X = [[1, 91], [1, 109], [1, 91]]
Y = [[-90, 90], [-126, 90], [-72, 108]]
# test value
x_interp = interp1d(X[0], Y[0])
y_interp = interp1d(X[1], Y[1])
z_interp = interp1d(X[2], Y[2])
x_interp_bk = interp1d(Y[0], X[0])
y_interp_bk = interp1d(Y[1], X[1])
z_interp_bk = interp1d(Y[2], X[2])

In [67]:
## Test interpolation
pt = [[48,95,36],[48,25,47]]
print(pt)
pt_conv = [[x_interp(x).item(), y_interp(y).item(), z_interp(z).item()] for x,y,z in pt]
print(pt_conv)
## Interp back
pt_conv_bk = [[int(x_interp_bk(x).item()), int(y_interp_bk(y).item()), int(z_interp_bk(z).item())] for x,y,z in pt_conv]
print(pt_conv_bk)

[[48, 95, 36], [48, 25, 47]]
[[4.0, 62.0, -2.0], [4.0, -78.0, 20.0]]
[[48, 95, 36], [48, 25, 47]]


In [55]:
# pts_list_conj_conv = [[x_interp(x).item(), y_interp(y).item(), z_interp(z).item()] for x, y, z in pts_list_conj]

KeyboardInterrupt: 

In [69]:
# one_line, table = util_conv.find_structure(pts_list_conj_conv)
one_line, table = util_conv.find_structure(pt_conv)
one_line

Right Cerebrum
Frontal Lobe
Superior Frontal Gyrus
White Matter
Frontal_Sup_R (aal)
Left Cerebrum
Frontal Lobe
Middle Frontal Gyrus
Gray Matter
brodmann area 9
Frontal_Inf_Oper_L (aal)
Left Cerebrum
Parietal Lobe
Inferior Parietal Lobule
Gray Matter
brodmann area 40
Parietal_Inf_L (aal)


[' // Right Cerebrum // Frontal Lobe // Superior Frontal Gyrus // White Matter // undefined // Frontal_Sup_R (aal)',
 ' // Left Cerebrum // Frontal Lobe // Middle Frontal Gyrus // Gray Matter // brodmann area 9 // Frontal_Inf_Oper_L (aal)',
 ' // Left Cerebrum // Parietal Lobe // Inferior Parietal Lobule // Gray Matter // brodmann area 40 // Parietal_Inf_L (aal)']

In [44]:
set_lst = [set() for x in range(7)]
for result in one_line:
    for index, el in enumerate(result.split("//")):
        if index == 0 or el.rstrip().lstrip() == 'undefined': continue
        set_lst[index].add(el)


In [46]:
display(set_lst)

[set(),
 {' Inter-Hemispheric ',
  ' Left Brainstem ',
  ' Left Cerebellum ',
  ' Left Cerebrum ',
  ' Right Brainstem ',
  ' Right Cerebellum ',
  ' Right Cerebrum '},
 {' Cerebellum Anterior Lobe ',
  ' Cerebellum Posterior Lobe ',
  ' Frontal Lobe ',
  ' Frontal-Temporal Space ',
  ' Limbic Lobe ',
  ' Midbrain ',
  ' Occipital Lobe ',
  ' Parietal Lobe ',
  ' Pons ',
  ' Sub-lobar ',
  ' Temporal Lobe '},
 {' Anterior Cingulate ',
  ' Caudate ',
  ' Cerebellar Lingual ',
  ' Cingulate Gyrus ',
  ' Claustrum ',
  ' Culmen ',
  ' Culmen of Vermis ',
  ' Cuneus ',
  ' Declive ',
  ' Extra-Nuclear ',
  ' Fastigium ',
  ' Fourth Ventricle ',
  ' Fusiform Gyrus ',
  ' Inferior Frontal Gyrus ',
  ' Inferior Parietal Lobule ',
  ' Inferior Temporal Gyrus ',
  ' Insula ',
  ' Lateral Ventricle ',
  ' Lentiform Nucleus ',
  ' Lingual Gyrus ',
  ' Medial Frontal Gyrus ',
  ' Middle Frontal Gyrus ',
  ' Middle Occipital Gyrus ',
  ' Middle Temporal Gyrus ',
  ' Orbital Gyrus ',
  ' Paracentral